In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve
from math import sqrt

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir('C:/Users/tmutengwa/Documents/Acadgild/DataSources')

In [4]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [5]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [6]:
ddf = df.copy()

In [7]:
ddf.isnull().sum()

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

In [8]:
df.shape

(183978, 42)

In [9]:
ddf.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,183978.0,91989.500000,53110.018250,1.0,45995.25,91989.5,137983.75,183978.0
player_fifa_api_id,183978.0,165671.524291,53851.094769,2.0,155798.00,183488.0,199848.00,234141.0
player_api_id,183978.0,135900.617324,136927.840510,2625.0,34763.00,77741.0,191080.00,750584.0
overall_rating,183142.0,68.600015,7.041139,33.0,64.00,69.0,73.00,94.0
potential,183142.0,73.460353,6.592271,39.0,69.00,74.0,78.00,97.0
crossing,183142.0,55.086883,17.242135,1.0,45.00,59.0,68.00,95.0
finishing,183142.0,49.921078,19.038705,1.0,34.00,53.0,65.00,97.0
heading_accuracy,183142.0,57.266023,16.488905,1.0,49.00,60.0,68.00,98.0
short_passing,183142.0,62.429672,14.194068,3.0,57.00,65.0,72.00,97.0
volleys,181265.0,49.468436,18.256618,1.0,35.00,52.0,64.00,93.0


In [10]:
import pandas_profiling

In [11]:
pandas_profiling.ProfileReport(ddf)

Number of variables,42
Number of observations,183978
Total Missing (%),0.5%
Total size in memory,59.0 MiB
Average record size in memory,336.0 B
Numeric,31
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,7


In [12]:
ddf.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [13]:
ddf.drop(labels = ['preferred_foot', 'attacking_work_rate',
       'defensive_work_rate','ball_control', 'sprint_speed',
       'standing_tackle', 'sliding_tackle',
       'gk_handling', 'gk_positioning',
       'gk_reflexes'], axis = 1, inplace = True)

In [14]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [15]:
ddf.isna().sum()

id                       0
player_fifa_api_id       0
player_api_id            0
date                     0
overall_rating         836
potential              836
crossing               836
finishing              836
heading_accuracy       836
short_passing          836
volleys               2713
dribbling              836
curve                 2713
free_kick_accuracy     836
long_passing           836
acceleration           836
agility               2713
reactions              836
balance               2713
shot_power             836
jumping               2713
stamina                836
strength               836
long_shots             836
aggression             836
interceptions          836
positioning            836
vision                2713
penalties              836
marking                836
gk_diving              836
gk_kicking             836
dtype: int64

In [16]:
ddf.dropna(axis = 0, inplace = True )

In [17]:
ddf.shape

(181265, 32)

In [18]:
ddf.isna().sum()

id                    0
player_fifa_api_id    0
player_api_id         0
date                  0
overall_rating        0
potential             0
crossing              0
finishing             0
heading_accuracy      0
short_passing         0
volleys               0
dribbling             0
curve                 0
free_kick_accuracy    0
long_passing          0
acceleration          0
agility               0
reactions             0
balance               0
shot_power            0
jumping               0
stamina               0
strength              0
long_shots            0
aggression            0
interceptions         0
positioning           0
vision                0
penalties             0
marking               0
gk_diving             0
gk_kicking            0
dtype: int64

In [19]:
pandas_profiling.ProfileReport(ddf)

Number of variables,33
Number of observations,181265
Total Missing (%),0.0%
Total size in memory,45.6 MiB
Average record size in memory,264.0 B
Numeric,31
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [20]:
ddf.drop(ddf[['id', 'player_api_id', 'player_fifa_api_id', 'date']], axis = 1, inplace =True)

In [21]:
ddf.columns

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'acceleration', 'agility',
       'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength',
       'long_shots', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'marking', 'gk_diving', 'gk_kicking'],
      dtype='object')

In [22]:
#Now, split the dataset into features and target variable as follows −
y = ddf.pop('overall_rating') # Target variable

X = ddf # Features

In [23]:
ddf.columns

Index(['potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'acceleration', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'gk_diving', 'gk_kicking'],
      dtype='object')

In [24]:
#Next, we will divide the data into train and test split. 
#The following code will split the dataset into 70% training data and 30% of testing data −

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [25]:
#Model Comparison & Selection & Validation

Model = [
        #GLM
    LogisticRegression(),
    LinearRegression(),
    
    #Trees    
    DecisionTreeRegressor(),
      
    ]





Model_list = []
Model_comparison = pd.DataFrame(columns = Model_list)


row_index = 0
for alg in Model:
    
    
    predicted = alg.fit(X_train, y_train).predict(X_test)
    Model_name = alg.__class__.__name__
    Model_comparison.loc[row_index,'Model Name'] = Model_name
    Model_comparison.loc[row_index, 'Model Train Accuracy'] = round(alg.score(X_train, y_train), 4)
    Model_comparison.loc[row_index, 'Model Test Accuracy'] = round(alg.score(X_test, y_test), 4)
    #Model_comparison.loc[row_index, 'Model Precission'] = precision_score(y_test, predicted)
    #Model_comparison.loc[row_index, 'Model Recall'] = recall_score(y_test, predicted)
    Model_comparison.loc[row_index, 'Model RMSE'] =  np.sqrt(mean_squared_error(y_test, predicted))






    row_index+=1
    
Model_comparison.sort_values(by = ['Model RMSE'], ascending = True, inplace = True)    
Model_comparison

,Model Name,Model Train Accuracy,Model Test Accuracy,Model RMSE
2,DecisionTreeRegressor,0.9993,0.9534,1.508929
1,LinearRegression,0.8324,0.8310,2.875110
0,LogisticRegression,0.1170,0.1124,4.197893
